In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import math
import bisect
import altair as alt
pd.options.mode.chained_assignment = None
from Bio import SeqIO

In [ ]:
# Scaling factor for entrapment calculations, based on the ratio on the number of entrapment peptide sequences and human peptide sequences in the database

scaling_factor = 1 + 3105275 / 3608159
scaling_factor

1.8606258759661092

In [ ]:
entrapment_seq_path = r"C:\Users\Alex\Documents\Proteomes\MBR_Proteomes_March_2024\EntrapmentProteinPeptideSequences_50percent.txt"
with open(entrapment_seq_path) as f:
    entrapment_seqs = f.readlines()

entrapment_seqs_set = set()
for seq in entrapment_seqs:
    entrapment_seqs_set.add(seq.replace("\n", ""))


In [ ]:
# Build a dictionary linking protein accessions to their species of origin (For MaxQuant analysis)
accession_dict = dict()

yeast_fasta_sequences = SeqIO.parse(open(r"C:\Users\Alex\Documents\Proteomes\MBR_Proteomes_March_2024\uniprot_SCerevisiae_6k_03_2024.fasta"),'fasta')
yeast_records = list(yeast_fasta_sequences)
for record in yeast_records:
    accession_dict[record.id.split('|')[1]] = 'Yeast'

human_fasta_sequences = SeqIO.parse(open(r"C:\Users\Alex\Documents\Proteomes\MBR_Proteomes_March_2024\uniprot_HSapiens_80k_03_2024.fasta"),'fasta')
human_records = list(human_fasta_sequences)
for record in human_records:
    accession_dict[record.id.split('|')[1]] = 'Human'

human_fasta_sequences = SeqIO.parse(open(r"C:\Users\Alex\Documents\Proteomes\MBR_Proteomes_March_2024\ConcatenatedHumanEntrapmentProteins_50percent.fasta"),'fasta')
human_records = list(human_fasta_sequences)
for record in human_records:
    accession_dict[record.id.split('|')[1]] = 'Human'
    accession_dict[record.id] = 'Human'

arabida_fasta_sequences = SeqIO.parse(open(r"C:\Users\Alex\Documents\Proteomes\MBR_Proteomes_March_2024\uniprot_Arabidopsis_40k_03_2024.fasta"),'fasta')
arabida_records = list(arabida_fasta_sequences)
for record in arabida_records:
    accession_dict[record.id.split('|')[1]] = 'Arabida'

entrapment_fasta_sequences = SeqIO.parse(open(r"C:\Users\Alex\Documents\Proteomes\MBR_Proteomes_March_2024\HumanEntrapmentDb.fasta"), 'fasta')
entrapment_records = list(entrapment_fasta_sequences)
for record in entrapment_records:
    accession_dict[record.id.split('|')[1]] = 'Arabida'
    
entrapment_formatted_seq = SeqIO.parse(open(r"\\bison.chem.wisc.edu\share\Users\AlexanderS_Bison\MBR Data\Proteomes\HumanEntrapmentDb_Formatted.fasta"), 'fasta')
entrapment_records = list(entrapment_fasta_sequences)
for record in entrapment_records:
    accession_dict[record.id.split('|')[1]] = 'Arabida'
    accession_dict[record] = 'Arabida'

ecoli_fasta_sequences = SeqIO.parse(open(r"C:\Users\Alex\Documents\Proteomes\MBR_Proteomes_March_2024\uniprot_Ecoli_4k_03_2024.fasta"),'fasta')
ecoli_records = list(ecoli_fasta_sequences)
for record in ecoli_records:
    accession_dict[record.id.split('|')[1]] = 'Ecoli'

In [ ]:
def get_fdp_old_flash(o_peaks_path, c_peaks_path, censored_psm_path, human_file_pattern = "_1x02nguL_",
                   foreign_species = 'Yeast', rt_delta = 0.25):

    # calculate the error rate for (presumably) native human transfers
    peaks = pd.read_csv(o_peaks_path, sep = '\t')
    
    peaks = peaks.loc[~peaks["Protein Group"].str.contains("DECOY")]
    peaks = peaks.loc[peaks["File Name"].str.contains(human_file_pattern)]
    peaks["Organism"] = peaks.apply(get_organisms, axis = 1)

    msms_detected_yeast_peaks = peaks.loc[(peaks["Peak Detection Type"] == "MSMS") &
                           (peaks["Organism"].str.contains(foreign_species))]
    msms_detected_yeast_peaks.drop_duplicates(subset = ["Full Sequence"], keep = 'first', inplace = True)
    msms_yeast_seqs = set(msms_detected_yeast_peaks["Full Sequence"].tolist())

    peaks = peaks.loc[(peaks["Peak Detection Type"] == "MBR")]

    human_peaks = peaks.loc[peaks["Organism"].str.contains('Human')]
    # Our entrapment database has a bunch of scrambled sequences appended to the end of the DB, which is why we're checking for them
    entrapment_peaks = human_peaks.loc[human_peaks["Full Sequence"].isin(entrapment_seqs_set)]
    human_peaks = human_peaks.loc[~human_peaks["Base Sequence"].isin(entrapment_seqs_set)]
    
    human_count = human_peaks.shape[0]
    arabida_count = entrapment_peaks.shape[0]

    yeast_peaks = peaks.loc[peaks["Organism"].str.contains(foreign_species)]
    yeast_peaks = yeast_peaks.loc[~yeast_peaks["Organism"].str.contains('Human')]
    yeast_count = yeast_peaks.shape[0]
    
    msms_yeast_count = sum(yeast_peaks["Full Sequence"].isin(msms_yeast_seqs))
    yeast_count = yeast_count - msms_yeast_count

    #arabida_peaks = peaks.loc[peaks["Organism"].str.contains("Arabida")]
    #arabida_count = arabida_peaks.shape[0]

    scaled_arabida = arabida_count * scaling_factor
    nper_results = get_nper_old_flash(censored_psm_path, o_peaks_path, c_peaks_path, rt_delta)
    scaled_nper = human_count * nper_results["error_rate"]

    total = human_count + yeast_count + arabida_count

    fper = 100 * yeast_count / total
    nper = 100 * scaled_nper / total
    fder = (100 - fper) * scaled_arabida / total

    false_discovery_proportion = fper + nper + fder

    results = dict(
        {
            "human": human_count,
            "foreign": yeast_count,
            "arabida": arabida_count,
            "total": total,
            "scaled_arabida": scaled_arabida,
            "scaled_nper": scaled_nper,
            "sensitivity": nper_results["sensitivity"],
            "fper": fper,
            "nper": nper,
            "fder": fder,
            "false_discovery_proportion": false_discovery_proportion
        }
    )
    return results

def get_organisms(row):
    accesions = row["Protein Group"].split(";")
    organisms = []
    for a in accesions:
        if a in accession_dict:
            organisms.append(accession_dict[a])
        elif ("ENTRAPMENT" in a):
            organisms.append("Arabida")
    return ";".join(organisms)

def get_nper_old_flash(censored_path, old_peak_path, new_peak_path, rt_delta = 0.25):
    # Read in the list of peptides that were censored
    censored_psms = pd.read_csv(censored_path, sep = '\t')
    # Select the ones that were quantified in the initial FlashLFQ analysis
    original_peaks = pd.read_csv(old_peak_path, sep = '\t')
    original_peaks = original_peaks[original_peaks["Peak RT Apex"] != "-"]
    censored_peaks = pd.merge(censored_psms, original_peaks, how = "inner", left_on=["File Name", "Full Sequence"], right_on=["File Name", "Full Sequence"])
    censored_peaks = censored_peaks[["File Name", "Full Sequence", "Peak RT Start", "Peak RT Apex", "Peak RT End", "Peak Charge", "Peak intensity"]]
    # This will allow for comparison later, as all the new peaks were take from files named in this way
    censored_peaks["File Name"] = censored_peaks["File Name"].astype(str) + "-censored" 

    # Merge the old and new results
    new_peaks = pd.read_csv(new_peak_path, sep = '\t')
    new_peaks = new_peaks.loc[(new_peaks["Peak Detection Type"] == "MBR")]
    new_peaks = new_peaks[["File Name", "Full Sequence", "Peak RT Start", "Peak RT Apex", "Peak RT End", "Peak Charge",  "Peak intensity", "MBR Score"]]
    peak_join = pd.merge(censored_peaks, new_peaks, how = "inner", left_on=["File Name", "Full Sequence"], right_on=["File Name", "Full Sequence"])

    peak_join_no_dup = peak_join.drop_duplicates(subset = ["File Name", "Full Sequence"], keep = 'first')
    censored_peaks_no_dup = censored_peaks.drop_duplicates(subset = ["File Name", "Full Sequence"], keep = 'first')
    sensitivity = peak_join_no_dup.shape[0] / censored_peaks_no_dup.shape[0]

    #compare the old and new results (here, 1 means the match, 0 means they dont)
    peak_join["Agreement"] = peak_join.apply(lambda x: check_overlap_time(x, rt_delta), axis = 1)

    if(peak_join.shape[0] == 0):
        return dict({
            "sensitivity": 0,
            "error_rate": 0} )

    peak_join.sort_values(by = 'MBR Score', ascending=False, inplace=True)
    peak_join["good_transfers"] = (peak_join["Agreement"] == 1).cumsum()
    peak_join["bad_transfers"] = (peak_join["Agreement"] == 0).cumsum()
    peak_join["error_rate"] = peak_join.apply(calculateErrorRate, axis = 1)

    results = dict(
        {
            "sensitivity": sensitivity,
            "error_rate": peak_join["error_rate"].iloc[-1]
        }

    )

    return results

# Native Peak Error Rate functions
def check_overlap_time(table, rt_delta = 0.25):
    try:
        pip_apex = float(table["Peak RT Apex_y"])
        ms_apex = float(table["Peak RT Apex_x"])
    except:
        return -1
    if(abs(pip_apex-ms_apex) < rt_delta):
        return 1
    else:
        return 0
    
    
def calculateErrorRate(table):
    return table["bad_transfers"] / (table["good_transfers"] + table["bad_transfers"])

In [ ]:
def get_fdp_flash(o_peaks_path, c_peaks_path, censored_psm_path, human_file_pattern = "_1x02nguL_",
                   foreign_species = 'Saccharomyces cerevisiae', q_value = 0.05, rt_delta = 0.5):

    peaks = pd.read_csv(o_peaks_path, sep = '\t')
    peaks = peaks.loc[peaks["File Name"].str.contains(human_file_pattern)]

    msms_detected_yeast_peaks = peaks.loc[(peaks["Peak Detection Type"] == "MSMS") &
                           (peaks["Organism"].str.contains(foreign_species))]
    msms_detected_yeast_peaks.drop_duplicates(subset = ["Full Sequence"], keep = 'first', inplace = True)
    msms_yeast_seqs = msms_detected_yeast_peaks["Full Sequence"].tolist()

    msms_peaks = peaks.loc[(peaks["Peak Detection Type"] == "MSMS") &
                           (peaks["Decoy Peptide"] == False) &
                           ((peaks["Organism"].str.contains(foreign_species)) | (peaks["Organism"].str.contains('Homo sapiens')) | (peaks["Organism"].str.contains('Arabidopsis')))]
    msms_count = msms_peaks.shape[0]

    peaks = peaks.loc[(peaks["Peak Detection Type"] == "MBR") & (peaks["PIP Q-Value"] < q_value) & (peaks["Decoy Peptide"] == False)]

    peaks = peaks.loc[(peaks["Random RT"] == False) & (peaks["Decoy Peptide"] == False)]

    human_peaks = peaks.loc[peaks["Organism"].str.contains('Homo sapiens')]
    # Our entrapment database has a bunch of scrambled sequences appended to the end of the DB, which is why we're checking for them
    entrapment_peaks = human_peaks.loc[human_peaks["Full Sequence"].isin(entrapment_seqs_set)]
    human_peaks = human_peaks.loc[~human_peaks["Base Sequence"].isin(entrapment_seqs_set)]
    
    human_count = human_peaks.shape[0]
    arabida_count = entrapment_peaks.shape[0]

    yeast_peaks = peaks.loc[peaks["Organism"].str.contains(foreign_species)]
    yeast_peaks = yeast_peaks.loc[~yeast_peaks["Organism"].str.contains('Homo sapiens')]
    yeast_count = yeast_peaks.shape[0]
    
    msms_yeast_count = sum(yeast_peaks["Full Sequence"].isin(msms_yeast_seqs))
    yeast_count = yeast_count - msms_yeast_count

    #arabida_peaks = peaks.loc[peaks["Organism"].str.contains("ENTRAPMENT")]
    #arabida_peaks = arabida_peaks.loc[~arabida_peaks["Organism"].str.contains(foreign_species)]
    #arabida_count = arabida_peaks.shape[0]

    score_threshold = peaks["PIP PEP"].max()

    # calculate the error rate for (presumably) native human transfers
    nper_results = get_native_peak_error_rate( 
                            o_peaks_path = o_peaks_path,
                            c_peaks_path = c_peaks_path,
                            censored_psm_path = censored_psm_path,
                            q_value_threshold = q_value,
                            score_threshold = score_threshold,
                            rt_delta = rt_delta)
    

    scaled_arabida = arabida_count * scaling_factor
    scaled_nper = human_count * nper_results["error_rate"]

    total = human_count + yeast_count + arabida_count
    total = max(1, total)

    fper = 100 * yeast_count / total
    nper = 100 * scaled_nper / total
    fder = (100-fper) * (scaled_arabida / total)

    false_discovery_proportion = fper + nper + fder

    results = dict(
        {
            "human": human_count,
            "foreign": yeast_count,
            "arabida": arabida_count,
            "total": total,
            "msms_count" : msms_count,
            "scaled_arabida": scaled_arabida,
            "scaled_nper": scaled_nper,
            "sensitivity": nper_results["sensitivity"],
            "fper": fper,
            "nper": nper,
            "fder": fder,
            "false_discovery_proportion": false_discovery_proportion
        }
    )
    return results

# Native Peak Error Rate functions
def check_overlap_time(table, rt_delta = 0.25):
    try:
        pip_apex = float(table["Peak RT Apex_y"])
        ms_apex = float(table["Peak RT Apex_x"])
    except:
        return -1
    if(abs(pip_apex-ms_apex) < rt_delta):
        return 1
    else:
        return 0
    
def calculateErrorRate(table):
    return table["bad_transfers"] / (table["good_transfers"] + table["bad_transfers"])

def get_native_peak_error_rate(o_peaks_path, c_peaks_path, censored_psm_path, q_value_threshold, score_threshold=10, rt_delta = 0.25):
    # Read in the list of peptides that were censored
    censored_psms = pd.read_csv(censored_psm_path, sep = '\t')
    # Select the ones that were quantified in the initial FlashLFQ analysis
    original_peaks = pd.read_csv(o_peaks_path, sep = '\t')
    original_peaks = original_peaks[original_peaks["Peak RT Apex"] != "-"]
    
    censored_peaks = pd.merge(censored_psms, original_peaks, how = "inner", left_on=["File Name", "Full Sequence"], right_on=["File Name", "Full Sequence"])
    censored_peaks = censored_peaks[["File Name", "Full Sequence", "Peak RT Start", "Peak RT Apex", "Peak RT End", "Peak Charge", "Peak intensity"]]

    # This will allow for comparison later, as all the new peaks were take from files named in this way
    censored_peaks["File Name"] = censored_peaks["File Name"].astype(str) + "-censored" 

    # Merge the old and new results
    new_peaks = pd.read_csv(c_peaks_path, sep = '\t')
    new_peaks = new_peaks.loc[(new_peaks["Peak Detection Type"] == "MBR") & (new_peaks["Random RT"] == False) & (new_peaks["Decoy Peptide"] == False)]
    new_peaks = new_peaks.loc[(new_peaks["PIP Q-Value"] < q_value_threshold)]
    #new_peaks = new_peaks.loc[(new_peaks["MBR Score"] > score_threshold)]
    new_peaks = new_peaks[["File Name", "Full Sequence", "Peak RT Start", "Peak RT Apex", "Peak RT End", "Peak Charge",  "Peak intensity", "PIP PEP", 'PIP Q-Value']]
    peak_join = pd.merge(censored_peaks, new_peaks, how = "inner", left_on=["File Name", "Full Sequence"], right_on=["File Name", "Full Sequence"])

    peak_join["Agreement"] = peak_join.apply(lambda x: check_overlap_time(x, rt_delta), axis = 1)
    peak_join.sort_values(by = "Agreement", ascending=False, inplace=True)

    #compare the old and new results (here, 1 means the match, 0 means they dont)
    peak_join.drop_duplicates(subset = ["File Name", "Full Sequence"], keep = 'first', inplace=True)
    censored_peaks.drop_duplicates(subset = ["File Name", "Full Sequence"], keep = 'first', inplace=True)
    sensitivity = peak_join.shape[0] / censored_peaks.shape[0]


    if(peak_join.shape[0] == 0):
        return dict({
            "sensitivity": 0,
            "error_rate": 0} )

    peak_join.sort_values(by = 'PIP Q-Value', ascending=True, inplace=True)
    peak_join["good_transfers"] = (peak_join["Agreement"] == 1).cumsum()
    peak_join["bad_transfers"] = (peak_join["Agreement"] == 0).cumsum()
    peak_join["error_rate"] = peak_join.apply(calculateErrorRate, axis = 1)

    results = dict(
        {
            "sensitivity": sensitivity,
            "error_rate": peak_join["error_rate"].iloc[-1]
        }
    )

    return results


In [ ]:
def get_fdp_fragger(o_folder = r"D:\GygiTwoProteome_PXD014415\IonQuant1Percent",
                      c_folder= r"D:\GygiTwoProteome_PXD014415\IonQuant1Percent_censored",
                      censored_psm_path = r"D:\GygiTwoProteome_PXD014415\CensoredDataFiles_fragger\CensoredPsms.tsv",
                      foreign_species = "YEAST",
                      human_file_pattern = "_human_90min_",
                      rt_diff_threshold = 0.5):
        
        rt_diff_threshold = rt_diff_threshold * 60 # Convert to seconds, which is how iq stores RT

        human_rep_indices = get_indices(o_folder, human_file_pattern)
        human_rep_indices_censored = get_indices(c_folder, human_file_pattern)
        
        ion_count = count_ions_by_species(o_folder + r"\combined_modified_peptide.tsv", match_indices = human_rep_indices, foreign_species = foreign_species)
        nper_results = analyze_nper(o_folder = o_folder, c_folder = c_folder, censored_psm_path = censored_psm_path, 
                                         human_rep_indices = human_rep_indices, human_rep_indices_censored = human_rep_indices_censored,
                                         rt_diff_threshold = rt_diff_threshold)
        
        scaled_arabida = ion_count["arabida"] * scaling_factor
        scaled_nper = ion_count["human"] * nper_results["specificity"]

        ion_count["scaled_arabida"] = scaled_arabida
        ion_count["scaled_nper"] = scaled_nper
        ion_count["specificity"] = nper_results["specificity"]
        ion_count["sensitivity"] = nper_results["sensitivity"]
        ion_count["nper_pct_diff"] = nper_results["intensity_diff"]

        ion_count["fper"] = 100 * ion_count["foreign"]  / ion_count["total"]
        ion_count["nper"] = 100 * ion_count["scaled_nper"]  / ion_count["total"]
        ion_count["fder"] = (100-ion_count["fper"]) * ion_count["scaled_arabida"]  / ion_count["total"]
        ion_count["false_discovery_proportion"] = ion_count["fper"] + ion_count["nper"] + ion_count["fder"]

        return ion_count
    
def get_indices(folder_path, pattern = "_human_90min_"):
    exp_file_path = folder_path + r"\experiment_annotation.tsv"
    exp_file = pd.read_csv(exp_file_path, sep = '\t')
    human_indices = exp_file[exp_file["file"].str.contains(pattern)].index
    return human_indices

def analyze_nper(o_folder,
                      c_folder,
                      censored_psm_path,
                      human_rep_indices,
                      human_rep_indices_censored = None,
                      rt_diff_threshold = 30):
    
    if(not any(human_rep_indices)):
        human_rep_indices_censored = human_rep_indices

    o_annotation_path = o_folder + r"\experiment_annotation.tsv"
    annotation_file = pd.read_csv(o_annotation_path, sep = '\t')
    file_dict_o = dict(zip(annotation_file.file, annotation_file.sample_name))
    original_file_dict = dict()
    for old_key in file_dict_o.keys():
        new_key = old_key.split("\\")[-1].split(".")[0]
        original_file_dict[new_key] = file_dict_o[old_key]

    c_annotation_path = c_folder + r"\experiment_annotation.tsv"
    annotation_file = pd.read_csv(c_annotation_path, sep = '\t')
    file_dict_c = dict(zip(annotation_file.file, annotation_file.sample_name))
    censored_file_dict = dict()
    for old_key in file_dict_c.keys():
        new_key = old_key.replace("-censored", "").split("\\")[-1].split(".")[0]
        censored_file_dict[new_key] = file_dict_c[old_key]

    # Read in the censored psms
    censored_psms = pd.read_csv(censored_psm_path, sep = '\t')
    censored_psms["File Name"] = censored_psms["Spectrum File"].apply(lambda x: x.replace("-censored", "").replace("interact-", "").split("\\")[-1].split(".")[0])
    # The mods got screwed up during writing the psms, so we fix that here
    censored_psms["Modified Peptide"] = censored_psms.apply(lambda x: x["Modified Peptide"].replace("43", "42.0106"), axis = 1)
    censored_psms["Modified Peptide"] = censored_psms.apply(lambda x: x["Modified Peptide"].replace("147", "15.9949"), axis = 1)
    censored_psms["Modified Peptide"] = censored_psms.apply(lambda x: x["Modified Peptide"].replace("C", "C[57.0215]"), axis = 1)

    # Read in the original ions
    o_ion_path = o_folder + r"\combined_ion.tsv"
    original_ions = pd.read_csv(o_ion_path, sep = '\t')

    info_cols = original_ions.columns[[0, 1, 7,8,11,12,13,14,15,16, 17]].to_list()

    match_cols = [col for col in original_ions.columns if "Match Type" in col]
    intensity_cols = [col for col in original_ions.columns if "Intensity" in col]
    rt_cols = [col for col in original_ions.columns if "Apex Retention Time" in col]

    match_cols = [match_cols[i] for i in human_rep_indices]
    rt_cols = [rt_cols[i] for i in human_rep_indices]
    intensity_cols = [intensity_cols[i] for i in human_rep_indices]

    original_ions = original_ions[info_cols + match_cols + rt_cols + intensity_cols]

    ion_melt = pd.melt(original_ions, id_vars = info_cols + match_cols + intensity_cols, value_vars=rt_cols, var_name="RT Column", value_name="RT")
    ion_melt["Match Type"] = ion_melt.apply(get_match_type, axis = 1)
    ion_melt["Intensity"] = ion_melt.apply(get_intensity, axis = 1)
    ion_melt.drop(match_cols + intensity_cols, axis = 1, inplace = True)
    ion_melt = ion_melt.loc[ion_melt["Match Type"] == "MS/MS"]

    original_file_rev_dict = dict((v, k) for k, v in original_file_dict.items())
    ion_melt["File Name"] = ion_melt.apply(lambda x: original_file_rev_dict[x["RT Column"].split(" ")[0]], axis = 1)
    o_ion_merge = pd.merge(left = ion_melt, right=censored_psms[["File Name", "Modified Peptide"]],
                            how = "inner", left_on = ["File Name", "Modified Sequence"], right_on = ["File Name", "Modified Peptide"])
    o_ion_merge = o_ion_merge.sort_values("Intensity", ascending=False).groupby(["Modified Sequence", "File Name"], as_index = False).first() # Keep only the highest intensity charge state


    #Read in the ions derived from the censored data
    # melt it to have one row per peak (need to group rt, intensity, etc.)
    c_ion_path = c_folder + r"\combined_ion.tsv"
    censored_ions = pd.read_csv(c_ion_path, sep = '\t')
    info_cols = censored_ions.columns[[0, 1, 7,8,11,12,13,14,15,16, 17]].to_list()

    match_cols = [col for col in censored_ions.columns if "Match Type" in col]
    intensity_cols = [col for col in censored_ions.columns if "Intensity" in col]
    rt_cols = [col for col in censored_ions.columns if "Apex Retention Time" in col]

    match_cols = [match_cols[i] for i in human_rep_indices_censored]
    rt_cols = [rt_cols[i] for i in human_rep_indices_censored]
    intensity_cols = [intensity_cols[i] for i in human_rep_indices_censored]

    censored_ions = censored_ions[info_cols + match_cols + rt_cols + intensity_cols]

    c_ion_melt = pd.melt(censored_ions, id_vars = info_cols + match_cols + intensity_cols, value_vars=rt_cols, var_name="RT Column", value_name="RT")
    c_ion_melt["Match Type"] = c_ion_melt.apply(get_match_type, axis = 1)
    c_ion_melt["Intensity"] = c_ion_melt.apply(get_intensity, axis = 1)
    c_ion_melt.drop(match_cols + intensity_cols, axis = 1, inplace = True)

    censored_file_rev_dict = dict((v, k) for k, v in censored_file_dict.items())
    c_ion_melt["File Name"] = c_ion_melt.apply(lambda x: censored_file_rev_dict[x["RT Column"].split(" ")[0]], axis = 1)
    c_ion_melt = c_ion_melt.loc[c_ion_melt["Match Type"] == "MBR"]
    c_ion_merge = pd.merge(left = c_ion_melt, right=censored_psms[["File Name", "Modified Peptide"]],
                            how = "right", left_on = ["File Name", "Modified Sequence"], right_on = ["File Name", "Modified Peptide"])
    c_ion_merge = c_ion_merge.sort_values("Intensity", ascending=False).groupby(["Modified Sequence", "File Name"], as_index = False).first() # Keep only the highest intensity charge state

    # Calculate the sensitivity (number of MBR ions / number of ions detected w/ MSMS)
    ion_comp = pd.merge(o_ion_merge, c_ion_merge, on = ["Modified Peptide", "File Name"], suffixes = ("_o", "_c"), how = "inner")
    ion_comp = ion_comp[ion_comp["RT_o"].notna() & ion_comp["RT_c"].notna()]
    ion_comp["RT_diff"] = abs(ion_comp["RT_o"] - ion_comp["RT_c"])
    ion_comp.sort_values("RT_diff", ascending=True, inplace=True)
    ion_comp = ion_comp.drop_duplicates(subset = ["Modified Peptide", "File Name"], keep = 'first')
    specificity = sum(ion_comp["RT_diff"] > rt_diff_threshold) / ion_comp.shape[0]

    ion_diffs = ion_comp.loc[ion_comp["RT_diff"] > rt_diff_threshold]
    pct_diffs = ion_diffs.apply(lambda x: 100 * abs(x["Intensity_o"] - x["Intensity_c"]) /
                                             ((x["Intensity_o"] + x["Intensity_c"]) / 2.0), axis = 1)
    

    o_ion_merge.drop_duplicates(subset = ["Modified Peptide", "File Name"], inplace = True)
    c_ion_merge.drop_duplicates(subset = ["Modified Peptide", "File Name"], inplace = True)
    number_of_ions_original = o_ion_merge.shape[0] - o_ion_merge["RT"].isna().sum()
    number_of_ions_censored = c_ion_merge.shape[0] - c_ion_merge["RT"].isna().sum()
    sensitivity = number_of_ions_censored / number_of_ions_original

    results = dict(sensitivity = sensitivity, specificity = specificity, intensity_diff = pct_diffs.mean())
    return results


# combined_ion parsing functions
def get_match_type(row):
    return(row[row["RT Column"].split(" ")[0] + " Match Type"])

def get_intensity(row):
    return(row[row["RT Column"].split(" ")[0] + " Intensity"])

def count_ions_by_species(path_to_combined_ion, match_indices = [0, 1, 2, 3, 4, 5, 6 ], foreign_species = "YEAST"):
    entrapment_species = "_ENT"

    # Read in combined ions
    ions = pd.read_csv(path_to_combined_ion, sep = '\t')
    # Keep informative columns (species, peptide, etc.) and "... Match Type" columns
    match_cols = [col for col in ions.columns if "Match Type" in col]
    # WARNING - This line is experiment dependent and should be changed based on the samples you wish to analyze
    match_cols = [match_cols[i] for i in match_indices] #Remove the two mixed proteome samples 
    info_cols = ions.columns[[0, 1, 7,8,11,12,13,14,15,16, 17]].to_list()
    ions = ions[info_cols + match_cols]
    # us melt to create one row per sample match type
    ion_mbr = ions.melt(info_cols)

    foreign_msms = ion_mbr.loc[(ion_mbr["Entry Name"].str.contains(foreign_species)) & (ion_mbr["value"] == "MS/MS")]
    foreign_msms_seqs = set(foreign_msms["Modified Sequence"].tolist())
    
    ion_msms = ion_mbr.loc[ion_mbr.value == "MS/MS"]
    ion_msms = ion_msms.loc[(ion_msms["Entry Name"].str.contains("HUMAN")) | (ion_msms["Entry Name"].str.contains(foreign_species)) | (ion_msms["Entry Name"].str.contains(entrapment_species))]
    msms_count = ion_msms.shape[0]

    # keep only the MBR match types
    ion_mbr = ion_mbr.loc[ion_mbr.value == "MBR"]

    ion_mbr.drop_duplicates(subset = ["Modified Sequence", "variable"], inplace = True)

    # Count the number of ions from each species
    human_ions = ion_mbr.loc[ion_mbr["Entry Name"].str.contains("HUMAN")]
    real_human_ions = human_ions.loc[~human_ions["Peptide Sequence"].isin(entrapment_seqs_set)]
    entrapment_ions = human_ions.loc[human_ions["Peptide Sequence"].isin(entrapment_seqs_set)]
    human_ion_count = real_human_ions.shape[0]
    arath_ion_count = entrapment_ions.shape[0]

    foreign_ions = ion_mbr.loc[ion_mbr["Entry Name"].str.contains(foreign_species)]
    foreign_ions["Mapped Proteins"] = foreign_ions["Mapped Proteins"].fillna('')
    foreign_ions = foreign_ions.loc[~foreign_ions["Mapped Proteins"].str.contains("HUMAN")]
    foreign_ion_count = foreign_ions.shape[0]

    foreign_double_count = sum(foreign_ions["Modified Sequence"].isin(foreign_msms_seqs))
    foreign_ion_count = foreign_ion_count - foreign_double_count



    count_dict = dict(
        {
            "human" : human_ion_count,
            "foreign" : foreign_ion_count,
            "arabida" : arath_ion_count,
            "total" : human_ion_count + foreign_ion_count + arath_ion_count,
            "msms_count" : msms_count,
            "mbr_count" : ion_mbr.shape[0]
        })
    return(count_dict)

In [ ]:
def get_fdp_maxquant(path, foreign_species = "Yeast", human_file_pattern = "_human_90min_"):
    
    ion_count = count_ions_by_species_mq(path, foreign_species, human_file_pattern)

    scaled_arabida = ion_count["arabida"] * scaling_factor
        
    ion_count["scaled_arabida"] = scaled_arabida

    ion_count["fper"] = 100 * ion_count["foreign"]  / ion_count["total"]
    ion_count["nper"] = 0
    ion_count["fder"] = (100-ion_count["fper"]) * ion_count["scaled_arabida"]  / ion_count["total"]
    ion_count["false_discovery_proportion"] = ion_count["fper"] + ion_count["nper"] + ion_count["fder"]

    return ion_count

def count_ions_by_species_mq(evidence_path = r"D:\Gygi_TwoProteomeData\combined_MaxQuant_Gygi\txt\evidence.txt", foreign_species = "Yeast",
                             human_file_pattern = "_human_90min_"):
    
    evidence = pd.read_csv(evidence_path, sep = '\t')
    evidence = evidence.loc[(evidence["Reverse"] != "+") & (evidence["Raw file"].str.contains(human_file_pattern))]
    evidence["Proteins"] = evidence["Proteins"].astype(str)
    evidence["Organism"] = evidence.apply(lambda x: ';'.join([get_species(protein) for protein in x['Proteins'].split(';')]), axis = 1)
    
    msms_foreign = evidence.loc[(evidence["MS/MS count"] > 0) & (evidence["Organism"].str.contains(foreign_species))]
    msms_foreign_seq = set(msms_foreign["Modified sequence"].tolist())

    pip = evidence.loc[evidence["Match score"].isna() == False]
    pip = pip.loc[pip["Raw file"].str.contains(human_file_pattern)]

    human_ions = pip.loc[pip["Organism"].str.contains("Human")]
    human_ion_real = human_ions.loc[~human_ions["Sequence"].isin(entrapment_seqs_set)]  
    entrapment_ions = human_ions.loc[human_ions["Sequence"].isin(entrapment_seqs_set)]  
    human_ion_count = human_ion_real.shape[0]
    arath_ion_count = entrapment_ions.shape[0]

    foreign_ions = pip.loc[pip["Organism"].str.contains(foreign_species)]
    foreign_ions = foreign_ions.loc[~foreign_ions["Organism"].str.contains("Human")]
    foreign_ion_count = foreign_ions.shape[0]

    foreign_ion_double_count = sum(foreign_ions["Modified sequence"].isin(msms_foreign_seq))
    foreign_ion_count = foreign_ion_count - foreign_ion_double_count

    #arath_ions = pip.loc[(pip["Organism"].str.contains("Arabida")) | (pip["Proteins"].str.contains("ENTRAPMENT"))]  
    #arath_ions = arath_ions.loc[~arath_ions["Organism"].str.contains("Human")]
    #arath_ions = arath_ions.loc[~arath_ions["Organism"].str.contains(foreign_species)]
    #arath_ion_count = arath_ions.shape[0]

    count_dict = dict(
        {
            "human" : human_ion_count,
            "foreign" : foreign_ion_count,
            "arabida" : arath_ion_count,
            "total" : sum([human_ion_count, foreign_ion_count, arath_ion_count])
        })
    return count_dict

def get_species(accession):
    species = accession_dict.get(accession, 'Other')
    return species

In [ ]:
# RT thresholds for native peak error calculations

gygi_rt = 90 * 0.01
inhouse_rt = 60 * 0.01
kelly_rt = 40 * 0.01

print("Gygi RT Threshold: ", gygi_rt)
print("Kelly RT Threshold: ", kelly_rt)
print("Inhouse RT Threshold: ", inhouse_rt)

Gygi RT Threshold:  0.9
Kelly RT Threshold:  0.4
Inhouse RT Threshold:  0.6


In [ ]:
gygi_flashv1 = get_fdp_old_flash(o_peaks_path = r"D:\GygiTwoProteome_PXD014415\FlashLFQ_CurRel_PepQ_ConcatenatedDB\QuantifiedPeaks.tsv",
                          c_peaks_path = r"D:\GygiTwoProteome_PXD014415\CensoredData_FlashLFQ_CurRel_PepQ_ConcatenatedDB\QuantifiedPeaks.tsv",
                          censored_psm_path = r"D:\GygiTwoProteome_PXD014415\MsConvertmzMLs\MM_NewPep_CensoredMzmls\CensoredPsms.psmtsv",
                          human_file_pattern="human_90", foreign_species='Yeast', rt_delta=gygi_rt)

inhouse_flashv1 = get_fdp_old_flash(o_peaks_path = r"D:\Human_Ecoli_TwoProteome_60minGradient\Human_FlashLFQ_CurRel_PepQ_ConcatenatedDB\QuantifiedPeaks.tsv",
                          c_peaks_path = r"D:\Human_Ecoli_TwoProteome_60minGradient\CensoredHuman_FlashLFQ_CurRel_PepQ_ConcatenatedDB\QuantifiedPeaks.tsv",
                          censored_psm_path = r"D:\Human_Ecoli_TwoProteome_60minGradient\RawData\MM_CensoredMzml_8_3_24\CensoredPsms.psmtsv",
                          human_file_pattern="Human_C18", foreign_species='Ecoli', rt_delta=inhouse_rt)

kelly_flashv1= get_fdp_old_flash(o_peaks_path = r"D:\Kelly_TwoProteomeData\FlashLFQ_CurRel_PepQ_ConcatenatedDB\QuantifiedPeaks.tsv",
                          c_peaks_path = r"D:\Kelly_TwoProteomeData\CensoredData_FlashLFQ_CurRel_PepQ_ConcatenatedDB\QuantifiedPeaks.tsv",
                          censored_psm_path = r"D:\Kelly_TwoProteomeData\MsConvertMzMls\MM_Censored_8_2_24\CensoredPsms.psmtsv",
                          human_file_pattern="_1x02nguL_", foreign_species='Yeast', rt_delta=kelly_rt)

C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\1976879677.py:5: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  peaks = pd.read_csv(o_peaks_path, sep = '\t')
C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\1976879677.py:79: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  original_peaks = pd.read_csv(old_peak_path, sep = '\t')
C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\1976879677.py:87: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  new_peaks = pd.read_csv(new_peak_path, sep = '\t')
C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\1976879677.py:5: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  peaks = pd.read_csv(o_peaks_path, sep = '\t')
C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\1976879677.py:79: DtypeWarning

In [ ]:
# Current FlashLFQ
gygi_censored_psms = r"D:\GygiTwoProteome_PXD014415\MsConvertmzMLs\MM_NewPep_CensoredMzmls\CensoredPsms.psmtsv"

gygi_dd_1 = get_fdp_flash(o_peaks_path = r"D:\GygiTwoProteome_PXD014415\FlashLFQ_7772_DonorPepQ_02\QuantifiedPeaks.tsv",
                          c_peaks_path = r"D:\GygiTwoProteome_PXD014415\CensoredData_FlashLFQ_7772_DonorPepQ_02\QuantifiedPeaks.tsv",
                          censored_psm_path = gygi_censored_psms,
                          human_file_pattern="human_90", foreign_species='Saccharomyces cerevisiae', q_value=0.01, rt_delta=gygi_rt)

gygi_dd_2p5 = get_fdp_flash(o_peaks_path = r"D:\GygiTwoProteome_PXD014415\FlashLFQ_7772_DonorPepQ_05\QuantifiedPeaks.tsv",
                            c_peaks_path = r"D:\GygiTwoProteome_PXD014415\CensoredData_FlashLFQ_7772_DonorPepQ_05\QuantifiedPeaks.tsv",
                            censored_psm_path = gygi_censored_psms,
                            human_file_pattern="human_90", foreign_species='Saccharomyces cerevisiae', q_value=0.025, rt_delta=gygi_rt)

gygi_dd_5 = get_fdp_flash(o_peaks_path = r"D:\GygiTwoProteome_PXD014415\FlashLFQ_7772_DonorPepQ_1\QuantifiedPeaks.tsv",
                          c_peaks_path = r"D:\GygiTwoProteome_PXD014415\CensoredData_FlashLFQ_7772_DonorPepQ_1\QuantifiedPeaks.tsv",
                          censored_psm_path = gygi_censored_psms,
                          human_file_pattern="human_90", foreign_species='Saccharomyces cerevisiae', q_value=0.05, rt_delta=gygi_rt)

inhouse_censored_psms = r"D:\Human_Ecoli_TwoProteome_60minGradient\RawData\MM_CensoredMzml_8_3_24\CensoredPsms.psmtsv"

inhouse_dd_1 = get_fdp_flash(o_peaks_path = r"D:\Human_Ecoli_TwoProteome_60minGradient\Human_FlashLFQ_7772_DonorPepQ_02\QuantifiedPeaks.tsv",
                          c_peaks_path = r"D:\Human_Ecoli_TwoProteome_60minGradient\CensoredHuman_FlashLFQ_7772_DonorPepQ_02\QuantifiedPeaks.tsv",
                          censored_psm_path = inhouse_censored_psms,
                          human_file_pattern="Human_C18", foreign_species='Escherichia coli', q_value=0.01, rt_delta=inhouse_rt)

inhouse_dd_2p5 = get_fdp_flash(o_peaks_path = r"D:\Human_Ecoli_TwoProteome_60minGradient\Human_FlashLFQ_7772_DonorPepQ_05\QuantifiedPeaks.tsv",
                          c_peaks_path = r"D:\Human_Ecoli_TwoProteome_60minGradient\CensoredHuman_FlashLFQ_7772_DonorPepQ_05\QuantifiedPeaks.tsv",
                          censored_psm_path = inhouse_censored_psms,
                          human_file_pattern="Human_C18", foreign_species='Escherichia coli', q_value=0.025, rt_delta=inhouse_rt)

inhouse_dd_5 = get_fdp_flash(o_peaks_path = r"D:\Human_Ecoli_TwoProteome_60minGradient\Human_FlashLFQ_7772_DonorPepQ_1\QuantifiedPeaks.tsv",
                          c_peaks_path = r"D:\Human_Ecoli_TwoProteome_60minGradient\CensoredHuman_FlashLFQ_7772_DonorPepQ_1\QuantifiedPeaks.tsv",
                          censored_psm_path = inhouse_censored_psms,
                          human_file_pattern="Human_C18", foreign_species='Escherichia coli', q_value=0.05, rt_delta=inhouse_rt)

kelly_censored_psms = r"D:\Kelly_TwoProteomeData\MsConvertMzMls\MM_Censored_8_2_24\CensoredPsms.psmtsv"

kelly_dd_1 = get_fdp_flash(o_peaks_path = r"D:\Kelly_TwoProteomeData\FlashLFQ_7772_DonorPepQ_02\QuantifiedPeaks.tsv",
                          c_peaks_path = r"D:\Kelly_TwoProteomeData\CensoredData_FlashLFQ_7772_DonorPepQ_02\QuantifiedPeaks.tsv",
                          censored_psm_path = kelly_censored_psms,
                          human_file_pattern="_1x02nguL_", foreign_species='Saccharomyces cerevisiae', q_value=0.01, rt_delta=kelly_rt)

kelly_dd_2p5 = get_fdp_flash(o_peaks_path = r"D:\Kelly_TwoProteomeData\FlashLFQ_7772_DonorPepQ_05\QuantifiedPeaks.tsv",
                          c_peaks_path = r"D:\Kelly_TwoProteomeData\CensoredData_FlashLFQ_7772_DonorPepQ_05\QuantifiedPeaks.tsv",
                          censored_psm_path = kelly_censored_psms,
                          human_file_pattern="_1x02nguL_", foreign_species='Saccharomyces cerevisiae', q_value=0.025, rt_delta=kelly_rt)


kelly_dd_5 = get_fdp_flash(o_peaks_path = r"D:\Kelly_TwoProteomeData\FlashLFQ_7772_DonorPepQ_1\QuantifiedPeaks.tsv",
                          c_peaks_path = r"D:\Kelly_TwoProteomeData\CensoredData_FlashLFQ_7772_DonorPepQ_1\QuantifiedPeaks.tsv",
                          censored_psm_path = kelly_censored_psms,
                          human_file_pattern="_1x02nguL_", foreign_species='Saccharomyces cerevisiae', q_value=0.05, rt_delta=kelly_rt)



C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\1540678269.py:4: DtypeWarning: Columns (10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  peaks = pd.read_csv(o_peaks_path, sep = '\t')
C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\1540678269.py:101: DtypeWarning: Columns (10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  original_peaks = pd.read_csv(o_peaks_path, sep = '\t')
C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\1540678269.py:111: DtypeWarning: Columns (10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  new_peaks = pd.read_csv(c_peaks_path, sep = '\t')
C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\1540678269.py:4: DtypeWarning: Columns (10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  peaks = pd.read_csv(o_peaks_path, sep = '\t')
C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\1540678269.py:101: DtypeWa

In [ ]:
gygi_iq_1 = get_fdp_fragger(o_folder = r"D:\GygiTwoProteome_PXD014415\MsConvertmzMLs\IonQuant_1Percent_50PercentEntrapmentDb",
                                c_folder= r"D:\GygiTwoProteome_PXD014415\MsConvertmzMLs\CensoredData_IonQuant_1Percent_50PercentEntrapmentDb",
                                censored_psm_path=r"D:\GygiTwoProteome_PXD014415\MsConvertmzMLs\CensoredFiles_Fragger_8_15_24\CensoredPsms.tsv",
                                human_file_pattern="human_90",
                                rt_diff_threshold=gygi_rt)

gygi_iq_2p5 = get_fdp_fragger(o_folder = r"D:\GygiTwoProteome_PXD014415\MsConvertmzMLs\IonQuant_2p5Percent_50PercentEntrapmentDb",
                                c_folder= r"D:\GygiTwoProteome_PXD014415\MsConvertmzMLs\CensoredData_IonQuant_2p5Percent_50PercentEntrapmentDb",
                                censored_psm_path=r"D:\GygiTwoProteome_PXD014415\MsConvertmzMLs\CensoredFiles_Fragger_8_15_24\CensoredPsms.tsv",
                                human_file_pattern="human_90",
                                rt_diff_threshold=gygi_rt) 

gygi_iq_5 = get_fdp_fragger(o_folder = r"D:\GygiTwoProteome_PXD014415\MsConvertmzMLs\IonQuant_5Percent_50PercentEntrapmentDb",
                                c_folder= r"D:\GygiTwoProteome_PXD014415\MsConvertmzMLs\CensoredData_IonQuant_5Percent_50PercentEntrapmentDb",
                                censored_psm_path=r"D:\GygiTwoProteome_PXD014415\MsConvertmzMLs\CensoredFiles_Fragger_8_15_24\CensoredPsms.tsv",
                                human_file_pattern="human_90",
                                rt_diff_threshold=gygi_rt)

inhouse_iq_1 = get_fdp_fragger(o_folder = r"D:\Human_Ecoli_TwoProteome_60minGradient\RawData\MzMls\IonQuant_1Percent_50PercentEntrapmentDb", 
                            c_folder= r"D:\Human_Ecoli_TwoProteome_60minGradient\RawData\MzMls\CensoredData_IonQuant_1Percent_50PercentEntrapmentDb",
                            censored_psm_path=r"D:\Human_Ecoli_TwoProteome_60minGradient\RawData\CensoredFiles_Fragger_8_16_24\CensoredPsms.tsv", 
                            human_file_pattern="_Human_C18_",
                            foreign_species="ECOLI",
                            rt_diff_threshold=inhouse_rt)

inhouse_iq_2p5 = get_fdp_fragger(o_folder = r"D:\Human_Ecoli_TwoProteome_60minGradient\RawData\MzMls\IonQuant_2p5Percent_50PercentEntrapmentDb", 
                            c_folder= r"D:\Human_Ecoli_TwoProteome_60minGradient\RawData\MzMls\CensoredData_IonQuant_2p5Percent_50PercentEntrapmentDb",
                            censored_psm_path=r"D:\Human_Ecoli_TwoProteome_60minGradient\RawData\CensoredFiles_Fragger_8_16_24\CensoredPsms.tsv", 
                            human_file_pattern="_Human_C18_",
                            foreign_species="ECOLI",
                            rt_diff_threshold=inhouse_rt)

inhouse_iq_5 = get_fdp_fragger(o_folder = r"D:\Human_Ecoli_TwoProteome_60minGradient\RawData\MzMls\IonQuant_5Percent_50PercentEntrapmentDb", 
                            c_folder= r"D:\Human_Ecoli_TwoProteome_60minGradient\RawData\MzMls\CensoredData_IonQuant_5Percent_50PercentEntrapmentDb",
                            censored_psm_path=r"D:\Human_Ecoli_TwoProteome_60minGradient\RawData\CensoredFiles_Fragger_8_16_24\CensoredPsms.tsv", 
                            human_file_pattern="_Human_C18_",
                            foreign_species="ECOLI",
                            rt_diff_threshold=inhouse_rt)

kelly_iq_1 = get_fdp_fragger(o_folder = r"D:\Kelly_TwoProteomeData\MsConvertMzMls\LibrarySettings_1Percent", 
                            c_folder= r"D:\Kelly_TwoProteomeData\MsConvertMzMls\LibrarySettings_CensoredData_1Percent",
                            censored_psm_path=r"D:\Kelly_TwoProteomeData\MsConvertMzMls\CensoredFiles_Fragger_8_12_24\CensoredPsms.tsv", 
                            human_file_pattern="_1x02nguL_",
                            rt_diff_threshold=kelly_rt)

kelly_iq_2p5 = get_fdp_fragger(o_folder = r"D:\Kelly_TwoProteomeData\MsConvertMzMls\LibrarySettings_2p5Percent_2", 
                            c_folder= r"D:\Kelly_TwoProteomeData\MsConvertMzMls\LibrarySettings_CensoredData_2p5Percent",
                            censored_psm_path=r"D:\Kelly_TwoProteomeData\MsConvertMzMls\CensoredFiles_Fragger_8_12_24\CensoredPsms.tsv", 
                            human_file_pattern="_1x02nguL_",
                            rt_diff_threshold=kelly_rt)

kelly_iq_5 = get_fdp_fragger(o_folder = r"D:\Kelly_TwoProteomeData\MsConvertMzMls\LibrarySettings_5Percent_2", 
                            c_folder= r"D:\Kelly_TwoProteomeData\MsConvertMzMls\LibrarySettings_CensoredData_5Percent",
                            censored_psm_path=r"D:\Kelly_TwoProteomeData\MsConvertMzMls\CensoredFiles_Fragger_8_12_24\CensoredPsms.tsv", 
                            human_file_pattern="_1x02nguL_",
                            rt_diff_threshold=kelly_rt)



In [ ]:
gygi_mq = get_fdp_maxquant(r"D:\GygiTwoProteome_PXD014415\combined_Gygi_8_16_Concatenated\txt\evidence.txt", "Yeast", human_file_pattern="human_90")

inhouse_mq = get_fdp_maxquant(r"D:\Human_Ecoli_TwoProteome_60minGradient\combined_Inhouse_8_16_Concatenated\txt\evidence.txt", "Ecoli", human_file_pattern="_Human_C18_")

kelly_mq = get_fdp_maxquant(r"D:\Kelly_TwoProteomeData\combined_Kelly_8_15_Concatenated\txt\evidence.txt", "Yeast", human_file_pattern="_1x02nguL_")

C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\4547196.py:19: DtypeWarning: Columns (51,52,59) have mixed types. Specify dtype option on import or set low_memory=False.
  evidence = pd.read_csv(evidence_path, sep = '\t')
C:\Users\Alex\AppData\Local\Temp\ipykernel_40240\4547196.py:19: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  evidence = pd.read_csv(evidence_path, sep = '\t')


In [ ]:
fdp_df = pd.DataFrame(
    {
        "Gygi_Flash_v1" : pd.Series(gygi_flashv1),
        "Gygi_Flash_v2_1" : pd.Series(gygi_dd_1),
        "Gygi_Flash_v2_2.5" : pd.Series(gygi_dd_2p5),
        "Gygi_Flash_v2_5" : pd.Series(gygi_dd_5),
        "Gygi_IonQuant_1" : pd.Series(gygi_iq_1),
        "Gygi_IonQuant_2.5" : pd.Series(gygi_iq_2p5),
        "Gygi_IonQuant_5" : pd.Series(gygi_iq_5),
        "Gygi_MaxQuant" : pd.Series(gygi_mq),

        "Kelly_Flash_v1" : pd.Series(kelly_flashv1),
        "Kelly_Flash_v2_1" : pd.Series(kelly_dd_1),
        "Kelly_Flash_v2_2.5" : pd.Series(kelly_dd_2p5),
        "Kelly_Flash_v2_5" : pd.Series(kelly_dd_5),
        "Kelly_IonQuant_1" : pd.Series(kelly_iq_1),
        "Kelly_IonQuant_2.5" : pd.Series(kelly_iq_2p5),
        "Kelly_IonQuant_5" : pd.Series(kelly_iq_5),
        "Kelly_MaxQuant" : pd.Series(kelly_mq),

        "Inhouse_Flash_v1" : pd.Series(inhouse_flashv1),
        "Inhouse_Flash_v2_1" : pd.Series(inhouse_dd_1),
        "Inhouse_Flash_v2_2.5" : pd.Series(inhouse_dd_2p5),
        "Inhouse_Flash_v2_5" : pd.Series(inhouse_dd_5),
        "Inhouse_IonQuant_1" : pd.Series(inhouse_iq_1),
        "Inhouse_IonQuant_2.5" : pd.Series(inhouse_iq_2p5),
        "Inhouse_IonQuant_5" : pd.Series(inhouse_iq_5),
        "Inhouse_MaxQuant" : pd.Series(inhouse_mq)
    }
)

fdp_df = fdp_df.transpose()
fdp_df["software"] = fdp_df.index

fdp_df.to_csv(r"C:\Users\Alex\Source\Repos\MBR_metamorpheus\R_Files\FDP_Analysis_Results.tsv", sep = '\t')

In [ ]:
# the following code was adapted from the following SO answer: https://stackoverflow.com/a/47040206
gygi_raw = pd.DataFrame(reversed([
    [int(7663*2.087), 572, 1680, 234, 428, int(gygi_mq_res["arabida"] * 2.087)],
    [4870, 407, 4535, 1498, 6154, int(gygi_mq_res["foreign"])],
    [1040, 230, 1003, 18118, 19649, 0]
]),
columns = ['FlashLFQ v1', 'FlashLFQ v2 1%', 'FlashLFQ v2 5%',  'IonQuant 1%', 'IonQuant 5%', "MaxQuant"],
index = reversed(["False Detection Errors", "Foreign Peak Errors", "Native Peak Errors"]))
gygi_raw

,FlashLFQ v1,FlashLFQ v2 1%,FlashLFQ v2 5%,IonQuant 1%,IonQuant 5%,MaxQuant
Native Peak Errors,1040,230,1003,18118,19649,0
Foreign Peak Errors,4870,407,4535,1498,6154,21831
False Detection Errors,15992,572,1680,234,428,1270


In [ ]:
# the following code was adapted from the following SO answer: https://stackoverflow.com/a/47040206
kelly_raw = pd.DataFrame(reversed([
    [int(773*2.087), 6, 198, int(kelly_1["arabida"] * 2.087), int(kelly_5["arabida"] * 2.087), int(kelly_mq["arabida"] * 2.087)],  #False Detection Errors
    [3558, 7, 1193, kelly_1['foreign'], kelly_5['foreign'], int(kelly_mq["foreign"])], # Foreign Peak Errors
    [228, 0, 325, 26, 386, 0] # Native Peak Errors
]),
columns = ['FlashLFQ v1', 'FlashLFQ v2 1%', 'FlashLFQ v2 5%',  'IonQuant 1%', 'IonQuant 5%', "MaxQuant"],
index = reversed(["False Detection Errors", "Foreign Peak Errors", "Native Peak Errors"]))
kelly_raw

,FlashLFQ v1,FlashLFQ v2 1%,FlashLFQ v2 5%,IonQuant 1%,IonQuant 5%,MaxQuant
Native Peak Errors,228,0,325,26,386,0
Foreign Peak Errors,3558,7,1193,66,1124,1127
False Detection Errors,1613,6,198,2,48,116


In [ ]:
inhouse_raw = pd.DataFrame(reversed([
    [12929, 438, 1200, int(inhouse_1["arabida"] * 2.087), int(inhouse_5["arabida"] * 2.087),
     int(inhouse_mq["arabida"] * 2.087)], #False Detection Errors
    [1940, 174, 1357, inhouse_1['foreign'], inhouse_5['foreign'],
     int(inhouse_mq["foreign"])], # Foreign Peak Errors
    [1239, 207, 1244, 5793, 6610, 0]  # Native Peak Errors
]),
columns = ['FlashLFQ v1', 'FlashLFQ v2 1%', 'FlashLFQ v2 5%',  'IonQuant 1%', 'IonQuant 5%', "MaxQuant"],
index = reversed(["False Detection Errors", "Foreign Peak Errors", "Native Peak Errors"]))
inhouse_raw

,FlashLFQ v1,FlashLFQ v2 1%,FlashLFQ v2 5%,IonQuant 1%,IonQuant 5%,MaxQuant
Native Peak Errors,1239,207,1244,5793,6610,0
Foreign Peak Errors,1940,174,1357,383,1080,3987
False Detection Errors,12929,438,1200,227,306,1333


In [ ]:
inhouse_pct = pd.DataFrame(reversed([
    [6.906, 0.621, 0.871, round(100*(inhouse_1["arabida"] * 2.087) / inhouse_1["total"], 3), round(100*(inhouse_5["arabida"] * 2.087) / inhouse_5["total"], 3),
     round(100* inhouse_mq["arabida"] * 2.087 / inhouse_mq["total"] ,3)], # False Detection
    [1.036, 0.246, 0.984, round(100*inhouse_1['foreign']/inhouse_1["total"], 3), round(100*inhouse_5['foreign']/inhouse_5["total"], 3),
     round(100* inhouse_mq["foreign"]/ inhouse_mq["total"] ,3)], # Foreign Peak
    [0.692, 0.295, 0.915, 5.44, 4.64, 0],  # Native Peak
]),
columns = ['FlashLFQ v1', 'FlashLFQ v2 1%', 'FlashLFQ v2 5%',  'IonQuant 1%', 'IonQuant 5%', "MaxQuant"],
index = reversed(["False Detection Errors", "Foreign Peak Errors", "Native Peak Errors"]))
inhouse_pct

,FlashLFQ v1,FlashLFQ v2 1%,FlashLFQ v2 5%,IonQuant 1%,IonQuant 5%,MaxQuant
Native Peak Errors,0.692,0.295,0.915,5.440,4.640,0.000
Foreign Peak Errors,1.036,0.246,0.984,0.358,0.826,1.627
False Detection Errors,6.906,0.621,0.871,0.212,0.235,0.544


In [ ]:
def prep_df(df, name):
    df = df.stack().reset_index()
    df.columns = ['Error Type', 'engine', 'values']
    df['DF'] = name
    return df

df1 = prep_df(gygi_raw, 'Lim et al.')
df2 = prep_df(inhouse_raw, 'E. coli')
df3 = prep_df(kelly_raw, 'Single Cell')

df = pd.concat([df1, df2, df3])

In [ ]:
df.sort_values(by = "Error Type", inplace=True)
df["Color"] = df["engine"] + df["Error Type"]

color = []
for x in df["Color"].tolist():
    if ("Foreign" in x):
        if('FlashLFQ v2 1%' in x):
            color.append('FlashLFQ v2 1%')
        elif('FlashLFQ v2 5%' in x):
            color.append('FlashLFQ v2 5%')
        elif("IonQuant 1%" in x):
            color.append('IonQuant 1%')
        elif("IonQuant 5%" in x):
            color.append('IonQuant 5%')
        elif('FlashLFQ v1' in x):
            color.append('FlashLFQ v1')
        elif('MaxQuant' in x):
            color.append('MaxQuant')
        else:
            color.append(x[0:8])
    else:
        color.append(x)

df["Software"] = color

In [ ]:
label_order = [ 
 'FlashLFQ v1', 'FlashLFQ v1False Detection Errors', 'FlashLFQ v1Native Peak Errors',
 'FlashLFQ v2 1%', 'FlashLFQ v2 1%False Detection Errors', 'FlashLFQ v2 1%Native Peak Errors',
 'FlashLFQ v2 5%', 'FlashLFQ v2 5%False Detection Errors', 'FlashLFQ v2 5%Native Peak Errors',
  'IonQuant 1%', 'IonQuant 1%False Detection Errors', 'IonQuant 1%Native Peak Errors',
  'IonQuant 5%', 'IonQuant 5%False Detection Errors', 'IonQuant 5%Native Peak Errors',
  'MaxQuant', 'MaxQuantFalse Detection Errors', 'MaxQuantNative Peak Errors'
  ]

alt.Chart(df).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('engine:N', title=None, sort = ["FlashLFQ v1", 'FlashLFQ v2 1%', 'FlashLFQ v2 5%', "IonQuant 1%", "IonQuant 5%", "MaxQuant"], axis=alt.Axis(labelAngle=-45)),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(values):Q', 
        axis=alt.Axis(
            grid=False,
            title="Number of False Detections")),

    # tell Altair which field to use to use as the set of columns to be  represented in each group
    column=alt.Column('DF:N', title=None, header=alt.Header(labelFontSize=24), spacing=45),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('Software:N', sort = label_order,
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['#13459c', '#2382f7','#a4cefc',
                       '#118C53', '#47B384','#B8E6DF',
                       '#118c11', '#47b347','#b8e6b8',
                       '#9f040e', '#e30613','#fc9ca2',
                       '#ff7b00', '#ffaa00','#ffe15c',
                       '#5A108F', '#AB51E3','#DC97FF'
                       ],
            ),
            legend=alt.Legend(values=["FlashLFQ v1", "FlashLFQ v2 1%", "FlashLFQ v2 5%", "IonQuant 1%", "IonQuant 5%", "MaxQuant"])
        ),
     
    order = alt.Order("Software:N", sort = 'ascending')
        )\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0   
    )\
        .properties(width = 200, height = 350)\
        .configure_legend(titleFontSize = 22, labelFontSize =18)\
        .configure_axis(titleFontSize = 20, labelFontSize =18)



alt.Chart(...)

In [ ]:
def prep_df(df, name):
    df = df.stack().reset_index()
    df.columns = ['Error Type', 'engine', 'values']
    df['DF'] = name
    return df

df1 = prep_df(gygi_pct, 'Lim et al.')
df2 = prep_df(inhouse_pct, 'E. coli')
df3 = prep_df(kelly_pct, 'Single Cell')

df = pd.concat([df1, df2, df3])

df.sort_values(by = "Error Type", inplace=True)
df["Color"] = df["engine"] + df["Error Type"]

color = []
for x in df["Color"].tolist():
    if ("Foreign" in x):
        if('FlashLFQ v2 2.5%' in x):
            color.append('FlashLFQ v2 2.5%')
        elif('FlashLFQ v2 5%' in x):
            color.append('FlashLFQ v2 5%')
        elif("IonQuant 1%" in x):
            color.append('IonQuant 1%')
        elif("IonQuant 5%" in x):
            color.append('IonQuant 5%')
        elif('FlashLFQ v1' in x):
            color.append('FlashLFQ v1')
        else:
            color.append(x[0:8])
    else:
        color.append(x)

df["Software"] = color

In [ ]:
label_order = [ 
 'FlashLFQ v1', 'FlashLFQ v1False Detection Errors', 'FlashLFQ v1Native Peak Errors',
 'FlashLFQ v2 2.5%', 'FlashLFQ v2 2.5%False Detection Errors', 'FlashLFQ v2 2.5%Native Peak Errors',
 'FlashLFQ v2 5%', 'FlashLFQ v2 5%False Detection Errors', 'FlashLFQ v2 5%Native Peak Errors',
  'IonQuant 1%', 'IonQuant 1%False Detection Errors', 'IonQuant 1%Native Peak Errors',
  'IonQuant 5%', 'IonQuant 5%False Detection Errors', 'IonQuant 5%Native Peak Errors',
  'MaxQuant', 'MaxQuantFalse Detection Errors', 'MaxQuantNative Peak Errors'
  ]

alt.Chart(df).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('engine:N', title=None, sort = ["FlashLFQ v1", 'FlashLFQ v2 2.5%', 'FlashLFQ v2 5%', "IonQuant 1%", "IonQuant 5%", "MaxQuant"], axis=alt.Axis(labelAngle=-45)),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(values):Q', 
        axis=alt.Axis(
            grid=False,
            title="Percent False Detections")),

    # tell Altair which field to use to use as the set of columns to be  represented in each group
    column=alt.Column('DF:N', title=None, header=alt.Header(labelFontSize=24), spacing=45),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('Software:N', sort = label_order,
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=['#13459c', '#2382f7','#a4cefc',
                       '#118C53', '#47B384','#B8E6DF',
                       '#118c11', '#47b347','#b8e6b8',
                       '#E54501', '#F48C06','#FAB82A',
                       '#9f040e', '#CC3943','#EA7981',
                       '#44485D', '#8D99AE','#A5B0C0'
                       ],
            ),
            legend=alt.Legend(values=["FlashLFQ v1", "FlashLFQ v2 2.5%", "FlashLFQ v2 5%", "IonQuant 1%", "IonQuant 5%", "MaxQuant"])
        ),
     
    order = alt.Order("Software:N", sort = 'ascending')
        )\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0   
    )\
        .properties(width = 400, height = 700)\
        .configure_legend(titleFontSize = 32, labelFontSize =18)\
        .configure_axis(titleFontSize = 32, labelFontSize =24)

alt.Chart(...)

In [ ]:
gygi_counts

,FlashLFQ v1,FlashLFQ v2 2.5%,FlashLFQ v2 5%,IonQuant 1%,IonQuant 5%,MaxQuant
PIP Identified Peak Traces,276230,158135,180523,364750,413139,255253


In [ ]:
prep_df_counts(gygi_counts, 'Lim et al.')

,counts,engine,values,DF
0,PIP Identified Peak Traces,FlashLFQ v1,276230,Lim et al.
1,PIP Identified Peak Traces,FlashLFQ v2 2.5%,158135,Lim et al.
2,PIP Identified Peak Traces,FlashLFQ v2 5%,180523,Lim et al.
3,PIP Identified Peak Traces,IonQuant 1%,364750,Lim et al.
4,PIP Identified Peak Traces,IonQuant 5%,413139,Lim et al.
5,PIP Identified Peak Traces,MaxQuant,255253,Lim et al.


In [ ]:
df1

,Error Type,engine,values,DF
0,PIP Identified Peak Traces,FlashLFQ v1,276230,Lim et al.
1,PIP Identified Peak Traces,FlashLFQ v2 2.5%,158135,Lim et al.
2,PIP Identified Peak Traces,FlashLFQ v2 5%,180523,Lim et al.
3,PIP Identified Peak Traces,IonQuant 1%,364750,Lim et al.
4,PIP Identified Peak Traces,IonQuant 5%,413139,Lim et al.
5,PIP Identified Peak Traces,MaxQuant,255253,Lim et al.


In [ ]:
def prep_df_counts(df, name):
    df = df.stack().reset_index()
    df.columns = ['counts', 'engine', 'values']
    df['DF'] = name
    return df

df1 = prep_df_counts(gygi_counts, 'Lim et al.')
df1["values"] = df1["values"] / 20
df2 = prep_df_counts(inhouse_counts, 'E. coli')
df2["values"] = df2["values"] / 10
df3 = prep_df_counts(kelly_counts, 'Single Cell')
df3["values"] = df3["values"] / 7

df = pd.concat([df1, df2, df3])

df["Color"] = df["engine"]

color = []
for x in df["Color"].tolist():
    if ("Foreign" in x):
        if('FlashLFQ v2 2.5%' in x):
            color.append('FlashLFQ v2 2.5%')
        elif('FlashLFQ v2 5%' in x):
            color.append('FlashLFQ v2 5%')
        elif("IonQuant 1%" in x):
            color.append('IonQuant 1%')
        elif("IonQuant 5%" in x):
            color.append('IonQuant 5%')
        elif('FlashLFQ v1' in x):
            color.append('FlashLFQ v1')
        else:
            color.append(x[0:8])
    else:
        color.append(x)

df["Software"] = color

In [ ]:
df

,counts,engine,values,DF,Color,Software
0,PIP Identified Peak Traces,FlashLFQ v1,13811.500000,Lim et al.,FlashLFQ v1,FlashLFQ v1
1,PIP Identified Peak Traces,FlashLFQ v2 2.5%,7906.750000,Lim et al.,FlashLFQ v2 2.5%,FlashLFQ v2 2.5%
2,PIP Identified Peak Traces,FlashLFQ v2 5%,9026.150000,Lim et al.,FlashLFQ v2 5%,FlashLFQ v2 5%
3,PIP Identified Peak Traces,IonQuant 1%,18237.500000,Lim et al.,IonQuant 1%,IonQuant 1%
4,PIP Identified Peak Traces,IonQuant 5%,20656.950000,Lim et al.,IonQuant 5%,IonQuant 5%
5,PIP Identified Peak Traces,MaxQuant,12762.650000,Lim et al.,MaxQuant,MaxQuant
0,PIP Identified Peak Traces,FlashLFQ v1,18045.200000,E. coli,FlashLFQ v1,FlashLFQ v1
1,PIP Identified Peak Traces,FlashLFQ v2 2.5%,4716.100000,E. coli,FlashLFQ v2 2.5%,FlashLFQ v2 2.5%
2,PIP Identified Peak Traces,FlashLFQ v2 5%,10621.700000,E. coli,FlashLFQ v2 5%,FlashLFQ v2 5%
3,PIP Identified Peak Traces,IonQuant 1%,10719.300000,E. coli,IonQuant 1%,IonQuant 1%


In [ ]:
label_order = [ 
 'FlashLFQ v1', 
 'FlashLFQ v2 2.5%', 
 'FlashLFQ v2 5%', 
  'IonQuant 1%', 
  'IonQuant 5%', 
  'MaxQuant'
  ]

alt.Chart(df).mark_bar().encode(

    # tell Altair which field to group columns on
    x=alt.X('engine:N', title=None, sort = ["FlashLFQ v1", 'FlashLFQ v2 2.5%', 'FlashLFQ v2 5%', "IonQuant 1%", "IonQuant 5%", "MaxQuant"], axis=alt.Axis(labelAngle=-45)),

    # tell Altair which field to use as Y values and how to calculate
    y=alt.Y('sum(values):Q', 
        axis=alt.Axis(
            grid=False,
            title="PIP Identifications Per File")),

    # tell Altair which field to use to use as the set of columns to be  represented in each group
    column=alt.Column('DF:N', title=None, header=alt.Header(labelFontSize=24), spacing=45),

    # tell Altair which field to use for color segmentation 
    color=alt.Color('Software:N', sort = label_order,
            scale=alt.Scale(
                # make it look pretty with an enjoyable color pallet
                range=[ '#13459c',
                       '#118C53',
                       '#118c11',
                       '#E54501',
                       '#9f040e',
                       '#44485D'
                       ],
            ),
            legend=alt.Legend(values=["FlashLFQ v1", "FlashLFQ v2 2.5%", "FlashLFQ v2 5%", "IonQuant 1%", "IonQuant 5%", "MaxQuant"])
        ),
     
    order = alt.Order("Software:N", sort = 'ascending')
        )\
    .configure_view(
        # remove grid lines around column clusters
        strokeOpacity=0   
    )\
        .properties(width = 200, height = 350)\
        .configure_legend(titleFontSize = 22, labelFontSize =18)\
        .configure_axis(titleFontSize = 20, labelFontSize =18)

alt.Chart(...)